In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

# Warnings
import warnings
warnings.filterwarnings('ignore')

## XGBoost

### Import libraries and data

In [2]:
import pandas as pd
import numpy as np

In [3]:
processed_test_df = pd.read_csv('../data/processed_test.csv')
processed_val_df = pd.read_csv('../data/processed_val.csv')

In [4]:
processed_test_df.head(3)

,mean_due_agencia,mean_due_canal,mean_due_ruta,mean_due_cliente,mean_due_prod_age,mean_due_prod_rut,mean_due_prod_cli,mean_due_prod_can,mean_due_prod_cli_age,mean_vh_agencia,std_due_acrcp,mean_due_acrcp
0,1.433683,1.529564,1.321099,1.179669,1.901905,1.630124,1.609438,2.068613,1.609438,3.295887,NaN,1.609438
1,1.440236,1.529564,1.392361,1.252450,1.017037,1.006477,NaN,1.192182,NaN,3.351941,NaN,NaN
2,1.552585,1.529564,1.511835,1.763140,1.527452,1.355031,1.098612,1.422284,1.098612,3.370756,0.0,1.098612


In [5]:
processed_val_df.head(3)

,Demanda_uni_equil,log_demanda_uni_equil,mean_due_agencia,mean_due_canal,mean_due_ruta,mean_due_cliente,mean_due_prod_age,mean_due_prod_rut,mean_due_prod_cli,mean_due_prod_can,mean_due_prod_cli_age,mean_vh_agencia,std_due_acrcp,mean_due_acrcp
0,1,0.693147,2.010012,1.996967,2.08378,1.719464,1.587694,1.556331,1.599232,1.512998,1.599232,4.113977,0.165949,1.599232
1,2,1.098612,2.010012,1.996967,2.08378,1.719464,1.680625,1.553152,1.212066,1.601887,1.212066,4.113977,0.320511,1.212066
2,2,1.098612,2.010012,1.996967,2.08378,1.719464,1.931563,1.803948,1.746179,1.819226,1.746179,4.113977,0.500848,1.746179


### Prepare data

In [6]:
é

### Build the model

In [10]:
from bimbo.xgboost_models import *

If you have the following error when you use XGboost: **"XGBoostError: [15:16:32] src/learner.cc:180: XGBoost version not compiled with GPU support."**

Please try the following:

`$ pip install pipdeptree
$ pipdeptree --reverse --packages xgboost`

In [11]:
model = build_model(X, y)

[23:09:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:09:47] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	validation_0-rmse:1.02133	validation_1-rmse:1.0217
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 1 rounds.
[1]	validation_0-rmse:0.790236	validation_1-rmse:0.790651
[2]	validation_0-rmse:0.645836	validation_1-rmse:0.646216
[3]	validation_0-rmse:0.561392	validation_1-rmse:0.561776
[4]	validation_0-rmse:0.513013	validation_1-rmse:0.513397
[5]	validation_0-rmse:0.488116	validation_1-rmse:0.488483
[6]	validation_0-rmse:0.474054	validation_1-rmse:0.474456
[7]	validation_0-rmse:0.466546	validation_1-rmse:0.466984
[8]	validation_0-rmse:0.46256	validation_1-rmse:0.463025
[9]	v

[131]	validation_0-rmse:0.449687	validation_1-rmse:0.45162
[132]	validation_0-rmse:0.44967	validation_1-rmse:0.451608
[133]	validation_0-rmse:0.449657	validation_1-rmse:0.451606
[134]	validation_0-rmse:0.449627	validation_1-rmse:0.451585
[135]	validation_0-rmse:0.449599	validation_1-rmse:0.451561
[136]	validation_0-rmse:0.449585	validation_1-rmse:0.451553
[137]	validation_0-rmse:0.449577	validation_1-rmse:0.451547
[138]	validation_0-rmse:0.449549	validation_1-rmse:0.451533
[139]	validation_0-rmse:0.449524	validation_1-rmse:0.451522
[140]	validation_0-rmse:0.449512	validation_1-rmse:0.451513
[141]	validation_0-rmse:0.449499	validation_1-rmse:0.451504
[142]	validation_0-rmse:0.449462	validation_1-rmse:0.45148
[143]	validation_0-rmse:0.449431	validation_1-rmse:0.451463
[144]	validation_0-rmse:0.449419	validation_1-rmse:0.451457
[145]	validation_0-rmse:0.449405	validation_1-rmse:0.451454
[146]	validation_0-rmse:0.449358	validation_1-rmse:0.451423
[147]	validation_0-rmse:0.449337	validation

### Save model to file

In [12]:
# save model to file
save_model(model, "model_2")

Saved model to: model_2.pickle.dat


### Submit to Kaggle

In [15]:
submit_df = submit_model(processed_test_df, model, "result_2")